In [15]:
from PyQt6.QtWidgets import *
import sys
import os
import json
from json import load
from datetime import datetime
from json import dumps,dump

In [16]:
#time stamp related routines
def get_timestamp_now():
    return int(datetime.timestamp( datetime.now() ))

def show_timestamp_as_string( ts):
    return datetime.ctime( datetime.fromtimestamp(ts) )

def time_test():
    tx = get_timestamp_now()
    print(show_timestamp_as_string(tx))

def keyfun( v ):
    return v['LastStudied']

def sort_subjects_on_time( d ):
    return sorted(d, key=keyfun, reverse=True)
    pass

In [17]:
def read_json(fname):
    with open(fname,"r") as fn:
        o = json.load(fn)
        return o

def write_json(newjs, dest):
    newFname = dest
    with open(newFname, 'w') as jsf:
        dump(newjs, jsf, indent=2)

In [18]:
class NewSubj:
    def __init__(self):
        self.layout = None
        self.show_subject_widgets = []
        self.add_subject_widgets = []
        self.sub_widgets = []
        #self.filename = '/mnt/d/thinking/thinking.json'
        self.filename  = 'D:\\thinking\\thinking.json'
        
    def clear_last_widgets(self):
        pass
    
    def call_user(self, subject):
        def subject_clicked():
            group_box = QGroupBox("Subject Details")
            self.clear_all()
            self.sub_widgets.append( group_box)
            #print(f" SUB:  {subject}")
            sub = QLabel()
            self.sub_widgets.append(sub)

            cat = QLabel()
            self.sub_widgets.append(cat)

            hrs = QLabel()
            self.sub_widgets.append(hrs)

            last = QLabel()
            self.sub_widgets.append(last)

            sub.setText(subject['Subject'] )
            cat.setText(subject['Category'])
            hrs.setText( str(subject['Hours']))
            last.setText(show_timestamp_as_string( subject['LastStudied']) )

            vbox = QVBoxLayout()
            self.sub_widgets.append( vbox )

            vbox.addWidget( sub  )
            vbox.addWidget( cat  )
            vbox.addWidget( hrs  )
            vbox.addWidget( last )

            group_box.setLayout( vbox )
            
            self.layout.addWidget( group_box)
            #self.layout.addRow(  vbox)
        return subject_clicked
           
    def addNewSubject(self):
        box = QGroupBox("New Subject")
        self.show_subject_widgets.append( box )
        flayout = QFormLayout()
        #self.show_subject_widgets.append( flayout) wrong hence commented
        self.sub = QLineEdit()
        self.show_subject_widgets.append(self.sub)
        self.typ = QLineEdit()
        self.show_subject_widgets.append(self.typ)
                       
        self.hours = QLineEdit()
        self.show_subject_widgets.append(self.hours)
        self.last = QLineEdit()
        self.show_subject_widgets.append(self.last)
        
        slabel =  QLabel("Subject")
        tlabel = QLabel("Category")         
        
        self.show_subject_widgets.append(slabel)
        self.show_subject_widgets.append(tlabel)               
        
        flayout.addRow( slabel, self.sub  )
        flayout.addRow( tlabel, self.typ  )        
              
        gb  = QPushButton("Submit")
        flayout.addWidget( gb)   
        box.setLayout(flayout)
        gb.clicked.connect(  self.submitClicked )
        self.layout.addWidget(box)
        self.lastWidget = box
        
        
    def submitClicked(self):
        sub = self.sub.text()
        d = { "Subject": sub,
            "Category" : self.typ.text(),           
            "Hours": 0,
            "LastStudied":  get_timestamp_now()
            }       
        
        inj = read_json(self.filename)
        inj["subjects"].append( d )
        write_json( inj, self.filename)
        self.selectionchange( 'menu')
        #self.lastWidget.deleteLater()  is this required, not sure     
     
    def clear_shown_subjects(self):
        for w in reversed( self.show_subject_widgets):
            #w.setParent(None)
            w.deleteLater() #added now ananda
        self.show_subject_widgets = []
            
    def clear_added_subjects(self):
        for w in reversed( self.add_subject_widgets):
            #w.setParent(None)
            w.deleteLater() #added now ananda
        self.add_subject_widgets = []

    def clear_sub_widgets( self):
        for w in reversed( self.sub_widgets):
            #w.setParent(None)
            w.deleteLater() #added now ananda
        self.sub_widgets = []
        
    
    def clear_all(self):        
        self.clear_sub_widgets()
        self.clear_shown_subjects()
        self.clear_added_subjects()
        
    def show_subjects_as_grid(self):        
        self.clear_all()
        topics = read_json( self.filename )
       
        for t in topics['subjects']:
            b1 = QPushButton(t['Subject'])
            clr = 'yellow'
            b1.setStyleSheet(f"background-color: {clr};")
            #b1.clicked.connect(  self.subject_clicked )
            b1.clicked.connect(  self.call_user(t) )
            self.layout.addWidget(b1)
            self.show_subject_widgets.append(b1)
            
    def show_subjects_in_timeline(self):        
        self.clear_all()
        topics = read_json( self.filename )
        group_box = QGroupBox("Subjects...")
        self.show_subject_widgets.append( group_box)
        
        sorted_subjects = sort_subjects_on_time( topics['subjects'])
        
        vbox = QVBoxLayout()
        self.show_subject_widgets.append( vbox )
        for t in sorted_subjects:
            b1 = QPushButton(t['Subject'])
            vbox.addWidget(b1)
            clr = 'yellow'
            b1.setStyleSheet(f"background-color: {clr};")
            #b1.clicked.connect(  self.subject_clicked  )
            b1.clicked.connect(  self.call_user(t) )
            self.layout.addWidget(b1)
            self.show_subject_widgets.append(b1)
        group_box.setLayout( vbox )            
        self.layout.addWidget( group_box)
        
    def selectionchange(self, t):      
        self.clear_all()
        self.win.resize(120,44)
        self.win.setLayout(self.layout)
        self.win.show()
            
        if t == 'showGrid':        
            self.show_subjects_as_grid()            
        elif t == 'menu':   
            #self.combo.resize(98,22)
            self.win.resize(120,44)
            self.win.show()   
            pass
        elif t == 'addSubjects':           
            self.addNewSubject()     
        elif t == 'showTimewise':
            self.show_subjects_in_timeline()
            
    
    def create_app(self):
        pass
        
    def drop_down(self):       
        app = QApplication(sys.argv)
        win = QWidget()
        self.app = app
        self.win = win
        self.combo = None

        self.layout = QGridLayout()   
    
        b1 = QComboBox()
        b1.addItem("menu")
        b1.addItem("addSubjects")
        b1.addItem("showGrid")
        b1.addItem("showTimewise")
       
        b1.currentTextChanged.connect(self.selectionchange)
        self.layout.addWidget(b1,0,0,1,4)
        self.combo = b1

        win.setLayout(self.layout)
        win.show()
       
        
    def start_exec(self):
        sys.exit(self.app.exec())

ap = NewSubj()
ap.drop_down()
ap.start_exec()

SystemExit: 0

Application is created

MainWindow
LayOut

Widgets:-
Always live in a layout and should be attached to some layout.
Can contain child widgets, which inturn can have child widgets.
How the widgets show up in a layout depends on the layout property.


25/Aug/2021:

We want high end study and hobby system:

maybe we want

Last studied: Time stamp
show by category
show all as grid
show last N studied



In [12]:
import pickle
def pickle_test():
    l = [1,2,3,4]
    with open( 'D:\\thinking\\funfile', 'wb') as f:
        pickle.dump(l, f)
    with open( 'D:\\thinking\\funfile', 'rb') as f:
        y = pickle.load( f)
    print(y)